# Ejemplo de agente productivo asíncrono

## Introducción

En este notebook se mostrará la arquitectua de solución y depliegue para el ejemplo de Agente para ser aprovisionado en producción. Para pruebas se utiliza el notebook y cuando se requiere lanzar a producción se utilzar runner.py


### Detalles de la implementación


| Información         |   Detalles                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tipo de Agente          | ejemplo                                                                         |
| Framework utilizado  | Google ADK                                                                   |
| Modelo LLM           | Anthropic Claude Haiku 3 streaming                                                     |
| Tipo de autenticación | token |
| OpenAI compatible   | Si                                                                   |


### Arquitectura del agente

En la siguiente imagen se puede visualizar un diagrama de la arquitectura del agente

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Diagrama de despligue
En la siguiente imagen se encuetra el diagrama de despliegue para esta arquitectura.

<div style="text-align:left">
    <img src="images/architecture_runtime_despliegue.png" width="50%"/>
</div>


### Características principales

* Configruación de sampling variables
* Uso de pydantic para manejar tipos de datos 
* Uso de ADK de Google


# Sincronizar 

Se hace carga de dependencias y variables de entorno

In [2]:
!uv sync

Resolved 133 packages in 3ms
Audited 128 packages in 2ms


In [3]:
from dotenv import load_dotenv
from pathlib import Path

env_path = Path("./adk_example/.env")

load_dotenv(
    dotenv_path=env_path
)

True

# Agente

In [4]:
%%writefile adk_example/agent.py
from google.adk.agents.llm_agent import LlmAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.planners import PlanReActPlanner

import datetime
from zoneinfo import ZoneInfo

def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city for which to retrieve the weather report.

    Returns:
        dict: status and result or error msg.
    """
    if city.lower() == "new york":
        return {
            "status": "success",
            "report": (
                "The weather in New York is sunny with a temperature of 25 degrees"
                " Celsius (77 degrees Fahrenheit)."
            ),
        }
    else:
        return {
            "status": "error",
            "error_message": f"Weather information for '{city}' is not available.",
        }


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    if city.lower() == "new york":
        tz_identifier = "America/New_York"
    else:
        return {
            "status": "error",
            "error_message": (
                f"Sorry, I don't have timezone information for {city}."
            ),
        }

    tz = ZoneInfo(tz_identifier)
    now = datetime.datetime.now(tz)
    report = (
        f'The current time in {city} is {now.strftime("%Y-%m-%d %H:%M:%S %Z%z")}'
    )
    return {"status": "success", "report": report}


root_agent = LlmAgent(
    model=LiteLlm(model="bedrock/anthropic.claude-3-haiku-20240307-v1:0"),
    name="weather_and_time_agent",
    instruction="You are an agent that returns time and weather",
    planner=PlanReActPlanner(),
    tools=[get_weather, get_current_time]
)

Overwriting adk_example/agent.py


In [5]:
!adk web

/Users/deimagjas/qubits/agents-templates/.venv/lib/python3.13/site-packages/google/adk/cli/fast_api.py:179: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/Users/deimagjas/qubits/agents-templates/.venv/lib/python3.13/site-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [17887]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                         